In [1]:
!pip install pygame==2.0.0.dev10
!pip install qiskit
!pip install quantumrandom

In [2]:
## Adding Quantum To The Game Wormy

import random
import pygame
import sys
import copy
from pygame.locals import *

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute, Aer
import quantumrandom as qr

pygame 2.0.0.dev10 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
FPS = 30

SMALLBOXSIZE = 60
SMALLBOARDSIZE = 6
SMALLMAXLIFE = 10
EASY = 0

MEDIUMBOXSIZE = 20
MEDIUMBOARDSIZE = 17
MEDIUMMAXLIFE = 30
MEDIUM = 1

LARGEBOXSIZE = 11
LARGEBOARDSIZE = 30
LARGEMAXLIFE = 64
HARD = 2

WINDOWWIDTH = 640
WINDOWHEIGHT = 480

PALETTEGAPSIZE = 10
PALETTESIZE = 45

boxSize = MEDIUMBOXSIZE
difficulty = MEDIUM
maxLife = MEDIUMMAXLIFE
boardWidth = MEDIUMBOARDSIZE
boardHeight = MEDIUMBOARDSIZE

WHITE = (255, 255, 255)
DARKGRAY = (70, 70, 70)
BLACK = (0, 0, 0)

COLOR1 = (DARKGRAY, (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 128, 0), (255, 0, 255))
COLOR2 = (BLACK, (2, 181, 160), (255, 211, 92), (234, 262, 12), (138, 151, 71), (250, 131, 1), (1, 128, 181))
COLOR3 = (DARKGRAY, (243, 182, 177), (245, 166, 97), (100, 165, 187), (171, 173, 89), (255, 216, 89), (169, 221, 199))
COLOR4 = (BLACK, (104, 73, 71), (188, 172, 144), (80, 57, 49), (180, 176, 173), (124, 103, 74), (255, 250, 226))
COLOR5 = (WHITE, (96, 74, 110), (95, 135, 85), (0, 126, 135), (33, 64, 95), (133, 105, 128), (200, 75, 109))
COLOR6 = (BLACK, (195, 63, 132), (144, 77, 110), (227, 226, 79), (251, 218, 213), (0, 125, 109), (75, 196, 213))

COLORSCHEME = (COLOR1, COLOR2, COLOR3, COLOR4, COLOR5, COLOR6)

for i in range(len(COLORSCHEME)):
    assert len(COLORSCHEME[i])==7, 'COLOR%s does not have exactly 7 colors'%(i)
    
bgColor = COLORSCHEME[0][0]
paletteColors = COLORSCHEME[0][1:]


In [4]:
def main():
    global FPSCLOCK, DISPLAYSURF, BASICFONT
    
    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    DISPLAYSURF = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))
    BASICFONT = pygame.font.Font('freesansbold.ttf', 18)
    pygame.display.set_caption('Quantum Flood It')
    
    mousex, mousey, mainBoard, life, lastPaletteClicked = showStartScreen()
    
    
    while True:
        paletteClicked = None
        resetGame = False
        
        DISPLAYSURF.fill(bgColor)
        drawResetAndSettings()
        drawBoard(mainBoard)
        drawLifeMeter(life)
        drawPalettes()
        
        checkForQuit()
        for event in pygame.event.get():
            if event.type == K_r:
                resetGame = True
            elif event.type == K_s:
                showSettingsScreen()
            elif event.type == MOUSEBUTTONUP:
                mousex, mousey = event.pos
                paletteClicked = getColorOfPaletteAt(mousex, mousey)
        
        if paletteClicked != None and paletteClicked != lastPaletteClicked:
            lastPaletteClicked = paletteClicked
            floodAnimation(mainBoard, paletteClicked)
            life -= 1
            
            resetGame = False
            if hasWon(mainBoard):
                for i in range(4):
                    flashBorderAnimation(WHITE, mainBoard)
                resetGame = True
                pygame.time.wait(2000)
            elif life == 0:
                drawLifeMeter(0)
                pygame.display.update()
                pygame.time.wait(400)
                for i in range(4):
                    flashBoarderAnimation(BLACK, mainBoard)
                resetGame = True
                pygame.time.wait(2000)
            
        if resetGame:
            mainBoard = generateRandomBoard(boardWidth, boardHeight, difficulty)
            life = maxLife
            lastPaletteClicked = None
            
        pygame.display.update()
        FPSCLOCK.tick(FPS)



In [5]:
def showStartScreen():
    titleFont = pygame.font.Font('freesansbold.ttf', 150)
    titleSurf = titleFont.render('Loading!', True, (50, 128, 246))
    titleRect = titleSurf.get_rect()
    titleRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2)
    quantumSurf = BASICFONT.render('Quantum random is slow :(', True, (50, 128, 246))
    quantumRect = quantumSurf.get_rect()
    quantumRect.center = (WINDOWWIDTH / 2, WINDOWHEIGHT/2+100)
    
    DISPLAYSURF.fill((248, 185, 129))
  
    DISPLAYSURF.blit(titleSurf, titleRect)
    DISPLAYSURF.blit(quantumSurf,quantumRect)
        
    pygame.display.update()
    FPSCLOCK.tick(FPS)
        
    mousex = 0
    mousey = 0
    mainBoard = generateRandomBoard(boardWidth, boardHeight, difficulty)
    life = maxLife
    lastPaletteClicked = None
    return mousex, mousey, mainBoard, life, lastPaletteClicked

In [6]:
def checkForQuit():
    for event in pygame.event.get(QUIT):
        pygame.quit()
        sys.exit()
    for event in pygame.event.get(KEYUP):
        if event.key == K_ESCAPE:
            pygame.quit()
            sys.exit()
        pygame.event.post(event)
    


In [7]:
def hasWon(board):
    for x in range(boardWidth):
        for y in range(boardHeight):
            if board[x][y] != board[0][0]:
                return False
    return True

In [8]:
def checkForKeyPress():
    if len(pygame.event.get(QUIT))>0:
        terminate()
    
    keyUpEvents = pygame.event.get(KEYUP)
    if len(keyUpEvents)==0:
        return None
    if keyUpEvents[0].key == K_ESCAPE:
        terminate()
    return keyUpEvents

In [9]:
def showSettingsScreen():
    global difficulty, boxSize, boardHeight, maxLife, paletteColors, bgColor
    
    origDifficulty = difficulty
    origBoxSize = boxSize
    screenNeedsRedraw = True
    
    
    while True:
        if screenNeedsRedraw:
            DISPLAYSURF.fill(bgColor)
     
            settingsFont = pygame.font.Font('freesansbold.ttf', 50)
            gameSurf = settingsFont.render('Click on a Difficulty/Color palette/Size to select.', True, WHITE)
            gameRect = settingsFont.get_rect()
            
            easySurf = settingsFont.render('Easy', True, WHITE)
            easyRect = settingsFont.get_rect()
            mediumSurf = settingsFont.render('Medium', True, WHITE)
            mediumRect = settingsFont.get_rect()
            hardSurf = settingsFont.redner('Hard', True, WHITE)
            hardRect = settingsFont.get_rect()

            smallSurf = settingsFont.render('Small Board', True, WHITE)
            smallRect = settingsFont.get_rect()
            midsizeSurf = settingsFont.render('Medium Board', True, WHITE)
            midsizeRect = settingsFont.get_rect()
            largeSurf = settingsFont.redner('Large Board', True, WHITE)
            largeRect = settingsFont.get_rect()
            
            exitSurf = settingsFont.render('Exit', True, WHITE)
            exitRect = settingsFont.get_rect()
    
            gameRect.midtop = (WINDOWWIDTH / 2, 10)
            easyRect.midtop = (WINDOWWIDTH / 4, gameRect.Height + 10 + 50)
            mediumRect.midtop = (WINDOWWIDTH / 4, easyRect.Height + 10 + 25)
            hardRect.midtop = (WINDOWWIDTH / 4, mediumRect.Height + 10 + 25)
            smallRect.midtop = (WINDOWWIDTH / 4, hardRect.Height + 10 + 50)
            midsizeRect.midtop = (WINDOWWIDTH / 4, smallRect.Height + 10 + 25)
            largeRect.midtop = (WINDOWWIDTH / 4, midsizeRect.Height + 10 + 25)
            exitRect.midtop = (WINDOWWIDTH/2, 420)

    
    
            DISPLAYSURF.blit(gameSurf, gameRect)
            DISPLAYSURF.blit(easySurf, easyRect)
            DISPLAYSURF.blit(mediumSurf, mediumRect)
            DISPLAYSURF.blit(hardSurf, hardRect)
            DISPLAYSURF.blit(smallSurf, smallRect)
            DISPLAYSURF.blit(midsizeSurf, midsizeRect)
            DISPLAYSURF.blit(largeSurf, largeRect)
            DISPLAYSURF.blit(exitSurf, exitRect)
            
            if difficulty == EASY:
                pygame.draw.circle(DISPLAYSURF, BLUE, easyRect.midright, 5, 4)
            if difficulty == MEDIUM:
                pygame.draw.circle(DISPLAYSURF, BLUE, mediumRect.midright, 5, 4)
            if difficulty == HARD:
                pygame.draw.circle(DISPLAYSURF, BLUE, hardRect.midright, 5, 4)
                
            if boxSize == SMALLBOXSIZE:
                pygame.draw.circle(DISPLAYSURF, BLUE, smallRect.midright, 5, 4)
            if boxsize == MEDIUMBOXSIZE:
                pygame.draw.circle(DISPLAYSURF, BLUE, midsizeRect.midright, 5, 4)
            if boxsize == LARGEBOXSIZE:
                pygame.draw.circle(DISPLAYSURF, BLUE, largeRect.midright, 5, 4)
                
                    
            for i in range(len(COLORSCHEMES)):
                drawColorSchemeBoxes(500, i*60+30, i)
                
                
            pygame.display.update()
            
            
        screenNeedsRedraw = False
            
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == KEYUP:
                if event.key == K_ESCAPE:
                    return not(origDifficulty == difficulty and origBoxsize == boxSize)
            elif event.type == MOUSEBUTTONUP:
                screenNeedsRedraw = True
                mousex, mousey = event.pos
                if pygame.Rect(easyRect).collidepoint(mousex, mousey):
                    difficulty = EASY
                elif pygame.Rect(mediumRect).collidepoint(mousex, mousey):
                    difficulty = MEDIUM
                elif pygame.Rect(hardRect).collidepoint(mousex, mousey):
                    difficulty = HARD
                    
                elif pygame.Rect(smallRect).collidepoint(mousex, mousey):
                    boxSize = SMALLBOXSIZE
                    boardWidth = SMALLBOARDSIZE
                    boardHeight = SMALLBOARDSIZE
                    maxLife = SMALLMAXLIFE
                elif pygame.Rect(midsizeRect).collidepoint(mousex, mousey):
                    boxSize = MEDIUMBOXSIZE
                    boardWidth = MEDIUMBOARDSIZE
                    boardHeight = MEDIUMBOARDSIZE
                    maxLife = MEDIUMMAXLIFE
                elif pygame.Rect(largeRect).collidepoint(mousex, mousey):
                    boxSize = LARGEBOXSIZE
                    boardWidth = LARGEBOARDSIZE
                    boardHeight = LARGEBOARDSIZE
                    maxLife = LARGEMAXLIFE
                elif pygame.Rect(exitRect).collidepoint(mousex, mousey):
                    return not(origDifficulty==difficulty and origBoxSize==boxSize)
                
                for i in range(len(COLORSCHEMES)):
                    if pygame.Rect(500, 30+i*60, MEDIUMBOXSIZE*3, MEDIUMBOXSIZE*2).collidepoint(mousex, mousey):
                        bgColor = COLORSCHEMES[i][0]
                        paletteColors = COLORSCHEMES[i][1:]
                

In [10]:
def drawColorSchemeBoxes(x, y, schemeNum):
    for boxy in range(2):
        for boxx in range(3):
            pygame.draw.rect(DISPLAYSURF, COLORSCHEMES[schemeNum][3*boxy+boxx+1], (x+MEDIUMBOXSIZE*boxx,
                            y+MEDIUMBOXSIZE * boxy, MEDIUMBOXSIZE, MEDIUMBOXSIZE))
            if paletteColors == COLORSCHEMES[schemeNum][1:]:
                pygame.draw.circle(DISPLAYSURF, BLUE, (600, schemeNum*60+30), 5, 4)

In [11]:
def flashBorderAnimation(color, board, animationSpeed=30):
    origSurf = DISPLAYSURF.copy()
    flashSurf = pygame.Surface(DISPLAYSURF.get_size())
    flashSurf = flashSurf.convert_alpha()
    for start, end, step in ((0, 256, 1), (255, 0, -1)):
        for transparence in range(start, end, animationSpeed*step):
            DISPLAYSURF.blit(origSurf, (0, 0))
            r, g, b = color
            flashSurf.fill((r, g, b, transparency))
            DISPLAYSURF.blit(flashSurf, (0, 0))
            drawBoard(board)
            pygame.display.update()
            FPSCLOCK.tick(FPS)
    DISPLAYSURF.blit(origSurf, (0,  0))

In [12]:
def floodAnimation(board, paletteClicked, animationSpeed=25):
    origBoard = copy.deepcopy(board)
    floodFill(board, board[0][0], paletteClicked, 0, 0)
    
    for transparency in range(0, 255, animationSpeed):
        drawBoard(origBoard)
        drawBoard(board, transparency)
        pygame.display.update()
        pygame.display.update()
        FPSCLOCK.tick(FPS)
    

In [13]:
def generateRandomBoard(width, height, difficulty = MEDIUM):
    board = []
    for x in range(width):
        column = []
        for y in range(height):
            # change to random.randint for faster start time
            column.append(int(qr.randint(0, len(paletteColors)-1)))
        board.append(column)
        print('%x / 16' %(x+1))
    
    # set some boxes to same color as a neighbour
    
    if difficulty == EASY:
        if boxSize == SMALLBOXSIZE:
            boxesToChange = 100
        else:
            boxesToChange = 1500
    elif difficulty == MEDIUM:
        if boxSize == SMALLBOXSIZE:
            boxesToChange = 5
        else:
            boxesToChange = 200
    else:
        boxesToChange = 0
        
    for i in range(boxesToChange):
        # change to random.randint for faster start time
        x = int(qr.randint(1, width-2))
        y = int(qr.randint(1, height-2))
        direction = int(qr.randint(0, 3))
        if direction == 0:
            board[x-1][y] = board[x][y]
            board[x][y-1] = board[x][y]
        elif direction == 1:
            board[x+1][y] = board[x][y]
            board[x][y+1] = board[x][y]
        elif direction == 2:
            board[x][y-1] = board[x][y]
            board[x+1][y] = board[x][y]
        else:
            board[x][y+1] = board[x][y]
            board[x-1][y] = board[x][y]
    return board

In [14]:
def drawResetAndSettings():
    pressKeySurf = BASICFONT.render('Press s for settings and r for reset', True, WHITE)
    pressKeyRect = pressKeySurf.get_rect()
    pressKeyRect.topleft = (WINDOWWIDTH-200, WINDOWHEIGHT-30)
    DISPLAYSURF.blit(pressKeySurf, pressKeyRect)

In [15]:
def drawBoard(board, transparency = 255):
    tempSurf = pygame.Surface(DISPLAYSURF.get_size())
    tempSurf = tempSurf.convert_alpha()
    tempSurf.fill((0, 0, 0, 0))
    
    for x in range(boardWidth):
        for y in range(boardHeight):
            left, top = leftTopPixelCoordOfBox(x, y)
            r, g, b = paletteColors[board[x][y]]
            pygame.draw.rect(tempSurf, (r, g, b, transparency), (left, top, boxSize, boxSize))
    left, top = leftTopPixelCoordOfBox(0, 0)
    pygame.draw.rect(tempSurf, BLACK, (left-1, top-1, boxSize*boardWidth+1, boxSize*boardHeight+1), 1)
    DISPLAYSURF.blit(tempSurf, (0, 0))
        

In [16]:
def drawPalettes():
    numColors = len(paletteColors)
    xmargin = int((WINDOWWIDTH-((PALETTESIZE*numColors) + (PALETTEGAPSIZE*(numColors-1))))/2)
    for i in range(numColors):
        left = xmargin + (i*PALETTESIZE) + (i * PALETTEGAPSIZE)
        top = WINDOWHEIGHT - PALETTESIZE - 10
        pygame.draw.rect(DISPLAYSURF, paletteColors[i], (left, top, PALETTESIZE, PALETTESIZE))
        pygame.draw.rect(DISPLAYSURF, bgColor, (left+2, top+2, PALETTESIZE-4, PALETTESIZE-4), 2)

In [17]:
def drawLifeMeter(currentLife):
    lifeBoxSize = int((WINDOWHEIGHT - 40)/maxLife)
    pygame.draw.rect(DISPLAYSURF, bgColor, (20, 20, 20, 20+(maxLife*lifeBoxSize)))
    for i in range(maxLife):
        if currentLife >= (maxLife-i):
            pygame.draw.rect(DISPLAYSURF, (255, 0, 0), (20, 20 + (i*lifeBoxSize), 20, lifeBoxSize))
        pygame.draw.rect(DISPLAYSURF, WHITE, (20, 20 + (i * lifeBoxSize),  20, lifeBoxSize), 1)

In [18]:
def getColorOfPaletteAt(x, y):
    numColors = len(paletteColors)
    xmargin = int((WINDOWWIDTH - (PALETTESIZE * numColors) + (PALETTEGAPSIZE * (numColors-1)))/2)
    top = WINDOWHEIGHT - PALETTESIZE - 10
    for i in range(numColors):
        left = xmargin + (i*PALETTESIZE) + (i*PALETTEGAPSIZE)
        r = pygame.Rect(left, top, PALETTESIZE, PALETTESIZE)
        if r.collidepoint(x, y):
            return i
    return None

In [19]:
def floodFill(board, oldColor, newColor, x, y):
    if oldColor == newColor or board[x][y]!= oldColor:
        return
    
    board[x][y] = newColor
    if x>0:
        floodFill(board, oldColor, newColor, x-1, y)
    if x < boardWidth-1:
        floodFill(board, oldColor, newColor, x+1, y)
    if y > 0:
        floodFill(board, oldColor, newColor, x, y-1)
    if y < boardHeight - 1:
        floodFill(board, oldColor, newColor, x, y+1)

In [20]:
def leftTopPixelCoordOfBox(boxx, boxy):
    xmargin = int((WINDOWWIDTH-(boardWidth * boxSize))/2)
    ymargin = int((WINDOWHEIGHT - (boardHeight * boxSize))/2)
    return (boxx* boxSize + xmargin, boxy * boxSize + ymargin)

In [ ]:
if __name__ == '__main__':
    main()

1 / 16
2 / 16
